# Reparación automática de circuitos reconfigurables

En primer lugar vamos a implementar una estructura de datos para el circuito. El método recibe como parámetros
    - m: número de capas del circuito
    - n: número de puertas en cada capa
    - programacionPuertas: lista con la programación de cada puerta
    - inputs: conexiones de cada puerta
    

In [1]:
def estructura_circuito(m,n, programacionPuertas, inputs):
    estructura = []
    for i in range (m):
        puertas = [];
        for j in range(n):
            puertas.append([programacionPuertas[(i*n)+j], inputs[(i*n)+j]]);
        estructura.append(puertas);
    return estructura;



A continuación se muestra un ejemplo de un circuito

In [2]:
programacionPuertasReal = ['AND', 'OR', '--', 'NOT', 'AND', 'OR','NOT', 'XOR', '--', '--', '--', 'NOT', 'NOT', 'AND', 'NAND']
conexionesReal = [[0,1], 
              [1,2],
              [2,3],
              [3,4],
              [2,4],
              [[0,0],[0,1]],
              [[0,0],[0,1]],
              [[0,1],[0,2]],
              [[0,3],[0,4]],
              [[0,3],[0,4]],
              [[0,0],[1,0]],
              [[1,0],[1,1]],
              [[1,1],[1,2]],
              [[1,2],[0,3]],
              [[1,3],[1,4]]]

ncapas = 3
npuertas = 5
circuitoReal = estructura_circuito(ncapas,npuertas, programacionPuertasReal, conexionesReal)

In [3]:
circuitoReal

[[['AND', [0, 1]],
  ['OR', [1, 2]],
  ['--', [2, 3]],
  ['NOT', [3, 4]],
  ['AND', [2, 4]]],
 [['OR', [[0, 0], [0, 1]]],
  ['NOT', [[0, 0], [0, 1]]],
  ['XOR', [[0, 1], [0, 2]]],
  ['--', [[0, 3], [0, 4]]],
  ['--', [[0, 3], [0, 4]]]],
 [['--', [[0, 0], [1, 0]]],
  ['NOT', [[1, 0], [1, 1]]],
  ['NOT', [[1, 1], [1, 2]]],
  ['AND', [[1, 2], [0, 3]]],
  ['NAND', [[1, 3], [1, 4]]]]]

En segundo lugar se implementa una función que actua como simulador de circuitos. 
La primera función determina la salida de una capa en función de los vectores salida de las dos capas anteriores. 
La segunda función va capa por capa llamando a la función anterior y devuelve la salida esperada si el circuito no estuviera defectuoso.
Por último la tercera función se encarga de obtener el generar un circuito contemplando las puertas defectuosas para poder pasarselo a la segunda función y obtener el vector de salida obtenido.

In [4]:
import copy
def resultado_circuito_capas (vector_entrada1, vector_entrada2, circuito, i, n):
    vector_salida = []
    if(i == 0):
        for j in range(n):
            puerta = circuito[i][j][0]
            conexiones_iniciales = circuito[i][j][1]
            if(puerta == 'AND'):
                if(vector_entrada2[conexiones_iniciales[0]] + vector_entrada2[conexiones_iniciales[1]] == 2):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
                    
                
            if(puerta == 'OR'):
                if(vector_entrada2[conexiones_iniciales[0]] + vector_entrada2[conexiones_iniciales[1]] >= 1):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
                  
            if(puerta == 'NOT'):
                if(vector_entrada2[conexiones_iniciales[0]] == 0):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
                  
            if(puerta == 'NAND'):
                if(vector_entrada2[conexiones_iniciales[0]] + vector_entrada2[conexiones_iniciales[1]] == 2):
                    vector_salida.append(0)
                else:
                    vector_salida.append(1)
                
            if(puerta == 'XOR'):
                if(vector_entrada2[conexiones_iniciales[0]] + vector_entrada2[conexiones_iniciales[1]] == 1):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
               
            if(puerta == '--'):
                    vector_salida.append(0)     
    if(i > 0):
        for j in range(n):
            puerta = circuito[i][j][0]
            conexiones = circuito[i][j][1]
            valor1 = 0
            valor2 = 0
            if(puerta == 'AND'):
                if(conexiones[0][0] == i-2):
                    valor1 = vector_entrada1[conexiones[0][1]]
                else:
                    valor1 = vector_entrada2[conexiones[0][1]]
                if(conexiones[1][0] == i-2):
                    valor2 = vector_entrada1[conexiones[1][1]]
                else:
                    valor2 = vector_entrada2[conexiones[1][1]]
                if(valor1 + valor2 == 2):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
           
                
            if(puerta == 'OR'):
                if(conexiones[0][0] == i-2):
                    valor1 = vector_entrada1[conexiones[0][1]]
                else:
                    valor1 = vector_entrada2[conexiones[0][1]]
                if(conexiones[1][0] == i-2):
                    valor2 = vector_entrada1[conexiones[1][1]]
                else:
                    valor2 = vector_entrada2[conexiones[1][1]]
                if(valor1 + valor2 >= 1):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
                    
              
            if(puerta == 'NOT'):
                if(conexiones[0][0] == i-2):
                    valor1 = vector_entrada1[conexiones[0][1]]
                else:
                    valor1 = vector_entrada2[conexiones[0][1]]
                
                if(valor1 == 0):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
                    
                    
              
            if(puerta == 'NAND'):
                if(conexiones[0][0] == i-2):
                    valor1 = vector_entrada1[conexiones[0][1]]
                else:
                    valor1 = vector_entrada2[conexiones[0][1]]
                if(conexiones[1][0] == i-2):
                    valor2 = vector_entrada1[conexiones[1][1]]
                else:
                    valor2 = vector_entrada2[conexiones[1][1]]
                
                if(valor1 + valor2 == 2):
                    vector_salida.append(0)
                else:
                    vector_salida.append(1)
              
              
            if(puerta == 'XOR'):
                if(conexiones[0][0] == i-2):
                    valor1 = vector_entrada1[conexiones[0][1]]
                else:
                    valor1 = vector_entrada2[conexiones[0][1]]
                if(conexiones[1][0] == i-2):
                    valor2 = vector_entrada1[conexiones[1][1]]
                else:
                    valor2 = vector_entrada2[conexiones[1][1]]
                
                if(valor1 + valor2 == 1):
                    vector_salida.append(1)
                else:
                    vector_salida.append(0)
            
            if(puerta == '--'):
                    vector_salida.append(0)     
              
             
    return vector_salida;

def resultado_circuito (vector_entrada1, vector_entrada2, circuito, m, n):
    vector_salida = []
    for i in range(m):
        vector_salida = resultado_circuito_capas (vector_entrada1, vector_entrada2, circuito, i, n)
        vector_entrada1 = vector_entrada2
        vector_entrada2 = vector_salida
    return vector_entrada2

def circuito_defectuoso (puertas_defectuosas, programacionPuertas, conexiones, ncapas, npuertas):
    circuito = estructura_circuito(ncapas,npuertas, programacionPuertas, conexiones)
    for i in range(len(puertas_defectuosas)):
        circuito[puertas_defectuosas[i][0]][puertas_defectuosas[i][1]][0] = '--'
        
    return circuito

Utilizando el ejemplo anterior calculamos el vector de salida correcto y el obtenido si el circuito estuviera defectuoso

In [27]:
vector_entrada1 = [0,0,0,0,0] # El vector de entrada uno es 0 al inicio del cálculo
#vector_entrada2 = [0,0,0,0,0]
vector_entrada2 = [1,1,1,1,1] # Entradas del circuito
puertas_defectuosas = [[1,0]]

vector_salida = resultado_circuito (vector_entrada1.copy(), vector_entrada2.copy(), circuitoReal, ncapas, npuertas)
circuitoDefectuoso = circuito_defectuoso(puertas_defectuosas.copy(), programacionPuertasReal, conexionesReal, ncapas, npuertas)
vector_salida_defectuoso = resultado_circuito(vector_entrada1.copy(), vector_entrada2.copy(), circuitoDefectuoso, ncapas, npuertas)
print("Vector de salida: ")
print(vector_salida)
print("Vector de salida defectuoso: ")
print(vector_salida_defectuoso)


Vector de salida: 
[0, 0, 1, 0, 1]
Vector de salida defectuoso: 
[0, 1, 1, 0, 1]


Vamos a implementar un algoritmo genético que nos encuentre buenos circuitos. En primer lugar importamos los módulos necesarios.

In [6]:
import random
from deap import base, creator, tools, algorithms
import numpy

D:\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


A continuación vamos nuestra caja de herramientas y registar todas las funciones relativas a los individuos

In [7]:
creator.create('Fitness', base.Fitness, weights=(1.0,))
creator.create('Individuo', list, fitness = creator.Fitness)
caja_de_herramientas = base.Toolbox()
caja_de_herramientas.register('gen', random.randint, 0, 5)
caja_de_herramientas.register('individuo', tools.initRepeat,
                              container=creator.Individuo, func=caja_de_herramientas.gen, n=ncapas*npuertas)
caja_de_herramientas.register('población', tools.initRepeat,
                              container=list, func=caja_de_herramientas.individuo, n=10)

Podemos considerar que cada puerta puede ser programada de 6 formas:
    - No programada (--) que será representada con el gen 0.
    - AND que se representa con el gen 1.
    - OR que se representa con el gen 2.
    - NOT que se representa con el gen 3.
    - XOR que se representa con el gen 4.
    - NAND que se representa con el gen 5.

In [8]:
random.seed(12345)  # Semilla para el mecanismo de generación de números aleatorios
for _ in range(5):
    print(caja_de_herramientas.gen())

3
5
0
2
2


In [9]:
random.seed(12345)
caja_de_herramientas.individuo()

[3, 5, 0, 2, 2, 1, 2, 4, 3, 1, 2, 0, 3, 2, 4]

In [10]:
random.seed(12345)
caja_de_herramientas.población()

[[3, 5, 0, 2, 2, 1, 2, 4, 3, 1, 2, 0, 3, 2, 4],
 [5, 1, 4, 4, 1, 2, 5, 5, 0, 4, 5, 3, 4, 4, 1],
 [1, 1, 5, 0, 1, 2, 2, 0, 3, 2, 0, 4, 5, 5, 3],
 [0, 0, 5, 1, 4, 1, 2, 0, 5, 3, 0, 5, 4, 1, 5],
 [3, 1, 4, 4, 5, 2, 0, 1, 0, 3, 1, 5, 4, 5, 1],
 [2, 4, 3, 4, 1, 0, 1, 2, 0, 1, 1, 2, 4, 2, 1],
 [3, 2, 0, 0, 0, 5, 3, 0, 3, 1, 0, 5, 2, 1, 5],
 [4, 0, 3, 4, 0, 0, 5, 3, 2, 4, 3, 4, 1, 2, 5],
 [5, 0, 4, 1, 0, 3, 1, 2, 1, 1, 1, 1, 2, 1, 4],
 [0, 4, 5, 0, 1, 0, 5, 3, 1, 4, 0, 2, 4, 3, 0]]

In [11]:
def fenotipo(individuo):
    programacionPuertas = []
    
    for i in range(ncapas*npuertas):
        if individuo[i] == 0:
            programacionPuertas.append('--')
        if individuo[i] == 1:
            programacionPuertas.append('AND')
        if individuo[i] == 2:
            programacionPuertas.append('OR')
        if individuo[i] == 3:
            programacionPuertas.append('NOT')
        if individuo[i] == 4:
            programacionPuertas.append('XOR')
        if individuo[i] == 5:
            programacionPuertas.append('NAND')
            
    return programacionPuertas

def evaluar_individuo(individuo):
    programacionPuertasAg = fenotipo(individuo)
    circuitoAG = estructura_circuito(ncapas,npuertas,  programacionPuertasAg, conexionesReal)
    circuitoReal = estructura_circuito(ncapas, npuertas, programacionPuertasReal, conexionesReal)
    salidaNueva = resultado_circuito(vector_entrada1.copy(), vector_entrada2.copy(), circuitoAG, ncapas, npuertas)
    salidaAntigua = resultado_circuito(vector_entrada1.copy(), vector_entrada2.copy(), circuitoReal, ncapas, npuertas)
    iguales = 0;
    
    for i in range (len(salidaAntigua)):
        if(salidaNueva == salidaAntigua):
            iguales = iguales + 1;
    return (iguales,)
    
caja_de_herramientas.register('evaluate', evaluar_individuo)    

In [12]:
evaluar_individuo([1,2,0,3,1,2,3,4,0,0,0,3,3,1,5])

(5,)

Ahora vamos a registrar los operadores a utilizar en el algoritmo genético, en concreto vamos a registrar los vistos en la práctica. También se añadirá como método de selección de individuos el método de selección por torneo.

In [13]:
caja_de_herramientas.register('mate', tools.cxOnePoint)
caja_de_herramientas.register('mutate', tools.mutFlipBit, indpb=0.1)
caja_de_herramientas.register('select', tools.selTournament, tournsize=3)

In [14]:
random.seed(12345)
P = caja_de_herramientas.población()
caja_de_herramientas.select(P, 2)

[[4, 0, 3, 4, 0, 0, 5, 3, 2, 4, 3, 4, 1, 2, 5],
 [0, 0, 5, 1, 4, 1, 2, 0, 5, 3, 0, 5, 4, 1, 5]]

In [20]:
random.seed(12345)
tamaño_vector_salida = 5
población_inicial = caja_de_herramientas.población()
población_final, registro = algorithms.eaSimple(población_inicial,
                                                caja_de_herramientas,
                                                cxpb=0.5,  # Probabilidad de cruzamiento
                                                mutpb=0.5,  # Probabilidad de mutación
                                                ngen=1000,  # Número de generaciones
                                                verbose=False)

for individuo in población_final:
    if(caja_de_herramientas.evaluate(individuo)[0] == tamaño_vector_salida):
        print(individuo, caja_de_herramientas.evaluate(individuo))



[0, 1, 0, 0, 0, 0, 3, 0, 1, 0, 1, 1, 2, 1, 5] (5,)
[0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 2, 1, 5] (5,)
[0, 1, 3, 1, 0, 1, 3, 0, 1, 0, 0, 0, 2, 1, 5] (5,)
[0, 1, 3, 1, 0, 1, 3, 0, 1, 0, 0, 0, 2, 1, 5] (5,)
[0, 1, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 2, 1, 5] (5,)
[0, 1, 0, 0, 0, 0, 3, 1, 1, 0, 1, 1, 2, 1, 5] (5,)
[0, 1, 0, 0, 0, 0, 3, 1, 1, 0, 1, 0, 2, 1, 5] (5,)
[0, 1, 0, 0, 0, 0, 3, 1, 1, 0, 1, 1, 2, 1, 5] (5,)


In [26]:
for individuo in población_final:
    programacionPuertasNuevas = []
    if(caja_de_herramientas.evaluate(individuo)[0] == tamaño_vector_salida):
        
        programacionPuertasNuevas = fenotipo(individuo)
        
        circuitoNuevo = estructura_circuito(ncapas,npuertas, programacionPuertasNuevas, conexionesReal)
        vector_salida = resultado_circuito (vector_entrada1.copy(), vector_entrada2.copy(), circuitoNuevo, ncapas, npuertas)
        circuitoNDefectuoso = circuito_defectuoso(puertas_defectuosas.copy(), programacionPuertasNuevas, conexionesReal, ncapas, npuertas)
        vector_salida_defectuoso = resultado_circuito(vector_entrada1.copy(), vector_entrada2.copy(), circuitoNDefectuoso, ncapas, npuertas)
        iguales = 0;
        for i in range(len(vector_salida)):
            iguales = iguales + abs(vector_salida[i] - vector_salida_defectuoso[i])
        if(iguales == 0):
            print(individuo)
            print(programacionPuertasNuevas)
            print("Vector de salida: ")
            print(vector_salida)
            print("Vector de salida defectuoso: ")
            print(vector_salida_defectuoso)


[0, 1, 0, 0, 0, 0, 3, 0, 1, 0, 1, 1, 2, 1, 5]
['--', 'AND', '--', '--', '--', '--', 'NOT', '--', 'AND', '--', 'AND', 'AND', 'OR', 'AND', 'NAND']
Vector de salida: 
[0, 0, 1, 0, 1]
Vector de salida defectuoso: 
[0, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 2, 1, 5]
['--', 'AND', '--', '--', '--', '--', 'NOT', 'AND', '--', '--', '--', '--', 'OR', 'AND', 'NAND']
Vector de salida: 
[0, 0, 1, 0, 1]
Vector de salida defectuoso: 
[0, 0, 1, 0, 1]
[0, 1, 3, 1, 0, 1, 3, 0, 1, 0, 0, 0, 2, 1, 5]
['--', 'AND', 'NOT', 'AND', '--', 'AND', 'NOT', '--', 'AND', '--', '--', '--', 'OR', 'AND', 'NAND']
Vector de salida: 
[0, 0, 1, 0, 1]
Vector de salida defectuoso: 
[0, 0, 1, 0, 1]
[0, 1, 3, 1, 0, 1, 3, 0, 1, 0, 0, 0, 2, 1, 5]
['--', 'AND', 'NOT', 'AND', '--', 'AND', 'NOT', '--', 'AND', '--', '--', '--', 'OR', 'AND', 'NAND']
Vector de salida: 
[0, 0, 1, 0, 1]
Vector de salida defectuoso: 
[0, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 2, 1, 5]
['--', 'AND', '--', '--', '--', '--', 'NOT', 'AND'